# Kaggle : Airbnb New User Bookings

In [1]:
from IPython.display import display
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import mglearn
import seaborn as sns

%matplotlib inline

from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression

In [2]:
import warnings
warnings.filterwarnings("ignore")

# data

# sessions

In [3]:
#10M data
sessions = pd.read_csv("data/sessions.csv")
sessions.head()

,user_id,action,action_type,action_detail,device_type,secs_elapsed
0,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,319.0
1,d1mm9tcy42,search_results,click,view_search_results,Windows Desktop,67753.0
2,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,301.0
3,d1mm9tcy42,search_results,click,view_search_results,Windows Desktop,22141.0
4,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,435.0


In [4]:
sessions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10567737 entries, 0 to 10567736
Data columns (total 6 columns):
user_id          object
action           object
action_type      object
action_detail    object
device_type      object
secs_elapsed     float64
dtypes: float64(1), object(5)
memory usage: 483.8+ MB


In [5]:
sessions.isnull().sum()

user_id            34496
action             79626
action_type      1126204
action_detail    1126204
device_type            0
secs_elapsed      136031
dtype: int64

In [6]:
#359 action type
sessions['action'].value_counts()

show                           2768278
index                           843699
search_results                  725226
personalize                     706824
search                          536057
ajax_refresh_subtotal           487744
update                          365130
similar_listings                364624
social_connections              339000
reviews                         320591
active                          188036
similar_listings_v2             168788
lookup                          162041
create                          155887
dashboard                       152952
header_userpic                  141830
collections                     124417
edit                            109083
campaigns                       105028
track_page_view                  81117
unavailabilities                 78317
qt2                              64651
notifications                    59392
confirm_email                    58726
requested                        57034
identity                 

In [7]:
sessions['action_type'].value_counts()

view                3560902
data                2103770
click               1996183
-unknown-           1031170
submit               623357
message_post          87103
partner_callback      19132
booking_request       18773
modify                 1139
booking_response          4
Name: action_type, dtype: int64

In [8]:
#155 action detail
sessions['action_detail'].value_counts()

view_search_results               1776885
p3                                1376550
-unknown-                         1031141
wishlist_content_update            706824
user_profile                       656839
change_trip_characteristics        487744
similar_listings                   364624
user_social_connections            336799
update_listing                     269779
listing_reviews                    269021
dashboard                          152952
user_wishlists                     152672
header_userpic                     141830
message_thread                     132395
edit_profile                       108686
message_post                        87103
contact_host                        81041
unavailable_dates                   77795
confirm_email_link                  58422
create_user                         54878
change_contact_host_dates           52517
user_profile_content_update         50434
user_reviews                        49665
p5                                

In [9]:
sessions['secs_elapsed'].describe()

count    1.043171e+07
mean     1.940581e+04
std      8.888424e+04
min      0.000000e+00
25%      2.290000e+02
50%      1.147000e+03
75%      8.444000e+03
max      1.799977e+06
Name: secs_elapsed, dtype: float64

# age_gender_bkts

In [10]:
#21 age bucket (5 years) per gender by country
#1K unit
age_gender = pd.read_csv("data/age_gender_bkts.csv")
age_gender.head()

,age_bucket,country_destination,gender,population_in_thousands,year
0,100+,AU,male,1.0,2015.0
1,95-99,AU,male,9.0,2015.0
2,90-94,AU,male,47.0,2015.0
3,85-89,AU,male,118.0,2015.0
4,80-84,AU,male,199.0,2015.0


In [11]:
age_gender.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 420 entries, 0 to 419
Data columns (total 5 columns):
age_bucket                 420 non-null object
country_destination        420 non-null object
gender                     420 non-null object
population_in_thousands    420 non-null float64
year                       420 non-null float64
dtypes: float64(2), object(3)
memory usage: 16.5+ KB


# countries (destination)

- AU = austrailia
- CA = canada
- DE = germany
- ES = spain
- FR = france
- GB = england
- IT = italia
- NL = netherland 
- PT = Portugal
- US = USA

In [ ]:
countries = pd.read_csv("data/countries.csv")
countries

In [ ]:
countries['country_destination'].value_counts()

# submission

In [ ]:
# submission = pd.read_csv("data/sample_submission_NDF.csv")
# submission.head()

In [ ]:
# submission.info()

# test

In [ ]:
test = pd.read_csv("data/test_users.csv")
test.head()

In [ ]:
test.isnull().sum()

# train

In [ ]:
train = pd.read_csv("data/train_users_2.csv")
train.head()

In [ ]:
train.isnull().sum()

# drop date_first_booking column

In [ ]:
train = train.drop(labels=['date_first_booking'], axis=1)
test = test.drop(labels=['date_first_booking'], axis=1)

# first_affiliate_tracked nan data manipulation

In [ ]:
train['first_affiliate_tracked'] = train.first_affiliate_tracked.replace(np.nan, "untracked")
test['first_affiliate_tracked'] = test.first_affiliate_tracked.replace(np.nan, "untracked")

# age

In [ ]:
print(len(train[train['age'] > 120]))

In [ ]:
train[train['age'] > 120]

In [ ]:
train_copy = train.copy()
train_copy = train_copy[train_copy['age'] <= 120]
train_copy.isnull().sum()

In [ ]:
train_copy = train.copy()
train_copy = train_copy.dropna()
train_copy = train_copy[train_copy['age'] <= 120]

ax1 = sns.distplot(train_copy['age'])
plt.show()
ax2 = sns.boxplot(x = train_copy['country_destination'], y = train_copy['age'], data = train_copy)
plt.show()
# ax3 = sns.swarmplot(x = train_copy['country_destination'], y = train_copy['age'], data = train_copy)
# plt.show()

# age nan value manipulation

- DBSCAN
- heatmap age vs related features

# pre processing

In [12]:
import pandas as pd

In [13]:
df_test_id = pd.read_csv("data/test_users.csv")

In [14]:
df_test_id = df_test_id['id']

In [15]:
df_sample = pd.read_csv('data/sample_submission_NDF.csv')

In [16]:
df_train = pd.read_csv("data/train_users_2.csv")
df_test = pd.read_csv("data/test_users.csv")
df_all = pd.concat([df_train, df_test])

In [17]:
df_target = df_train["country_destination"]
df_values = df_train["country_destination"].values

In [18]:
df_all = df_all.drop("age", axis = 1)
df_all = df_all.drop("date_first_booking", axis = 1)

In [19]:
df_all["first_affiliate_tracked"].fillna("untracked", inplace = True)

In [20]:
df_all.isnull().sum()

affiliate_channel              0
affiliate_provider             0
country_destination        62096
date_account_created           0
first_affiliate_tracked        0
first_browser                  0
first_device_type              0
gender                         0
id                             0
language                       0
signup_app                     0
signup_flow                    0
signup_method                  0
timestamp_first_active         0
dtype: int64

In [21]:
df_all["date_account_created"] = pd.to_datetime(df_all["date_account_created"], format = "%Y-%m-%d")

In [22]:
df_all["timestamp_first_active"] = pd.to_datetime(df_all["timestamp_first_active"], format="%Y%m%d%H%M%S", errors='ignore')

In [23]:
df_all["ac_year"] = df_all["date_account_created"].apply(lambda x : x.year)
df_all["ac_month"] = df_all["date_account_created"].apply(lambda x : x.month)
df_all["ac_day"] = df_all["date_account_created"].apply(lambda x : x.day)

In [24]:
df_all["fa_year"] = df_all["timestamp_first_active"].apply(lambda x : x.year)
df_all["fa_month"] = df_all["timestamp_first_active"].apply(lambda x : x.month)
df_all["fa_day"] = df_all["timestamp_first_active"].apply(lambda x : x.day)
df_all["fa_hour"] = df_all["timestamp_first_active"].apply(lambda x : x.hour)
df_all["fa_minute"] = df_all["timestamp_first_active"].apply(lambda x : x.minute)
df_all["fa_second"] = df_all["timestamp_first_active"].apply(lambda x : x.second)

In [25]:
df_all = df_all.drop("date_account_created", axis = 1)
df_all = df_all.drop("timestamp_first_active", axis = 1)
df_all = df_all.drop("country_destination", axis = 1)

In [26]:
df_all = df_all.drop("id", axis = 1)

In [27]:
df_all = pd.get_dummies(df_all)

In [28]:
df_train = df_all.iloc[:213451]
df_test = df_all.iloc[213451:]

In [29]:
df_train.shape, df_test.shape, df_target.shape

((213451, 145), (62096, 145), (213451,))

In [30]:
df_target.value_counts()

NDF      124543
US        62376
other     10094
FR         5023
IT         2835
GB         2324
ES         2249
CA         1428
DE         1061
NL          762
AU          539
PT          217
Name: country_destination, dtype: int64

In [31]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
y_label = le.fit_transform(df_target) 

In [32]:
y_label

array([ 7,  7, 10, ...,  7,  7,  7])

# age column

In [ ]:
test_age = pd.read_csv("last_test_data.csv")
train_age = pd.read_csv("last_train_data.csv")

test_age = test_age['age']
train_age = train_age['age']

In [ ]:
train_age.head(), len(train_age)

In [ ]:
test_age.head(), len(test_age)

In [ ]:
df_train = pd.concat([df_train, train_age], axis=1)
df_test = pd.concat([df_test, test_age], axis=1)

df_train.info(), df_test.info(), 

# age clustering

In [ ]:
# df_all = pd.concat([df_train, df_test], axis=0)
# df_all.head()


In [ ]:
# def pre_age_predict_data_cat():
    
#     bins = [0, 15, 25, 35, 60, 9999]
#     labels = ["미성년자", "청년", "중년", "장년", "노년"]
#     cats = pd.cut(df_all['age'], bins, labels=labels)
#     cats = pd.DataFrame(cats)
    
#     return cats

# cats=pre_age_predict_data_cat()

# cats_d = pd.get_dummies(cats)

# len(cats_d)

In [ ]:
# train_cat = cats_d[:213451]
# test_cat = cats_d[213451:]

# # df_train = df_train.drop("age", axis=1)
# # df_test = df_test.drop("age", axis=1)

# df_train = pd.concat([df_train, train_cat], axis=1)
# df_test = pd.concat([df_test, test_cat], axis=1)

# # df_train.to_csv('df_train.csv', sep=',', na_rep='NaN', index = False)
# # df_test.to_csv('df_test.csv', sep=',', na_rep='NaN', index = False)

# sessions

In [ ]:
sessions.head()

# Logistic Regression

In [33]:
from sklearn import linear_model

In [ ]:
logreg = linear_model.LogisticRegression(n_jobs=4, solver='newton-cg',random_state=0)

# we create an instance of Neighbours Classifier and fit the data.
model_log = logreg.fit(df_train, y_label)

In [ ]:
# y_pred_log = model_log.predict(df_test)
# y_inverse_log = le.inverse_transform(y_pred_log)
# y_inverse_log
# df_sample['country'] = y_inverse_log
# df_sample = df_sample.reset_index(drop=True)
# df_sample.to_csv('sub_log.csv', sep=',', na_rep='NaN', index = False)
#0.69443 #1200/1460

In [ ]:
#predict_proba
y_log1 = model_log.predict_proba(df_test)

ids = []  #list of ids
cts = []  #list of countries
for i in range(len(df_sample)):
    idx = df_sample.iloc[i, 0]
    ids += [idx] * 5
    cts += le.inverse_transform(np.argsort(y_log1[i])[::-1])[:5].tolist()

df_sample1 = pd.DataFrame(columns=['id', 'country'])
df_sample1['id'] = ids
df_sample1['country'] = cts
df_sample1 = df_sample1.reset_index(drop=True)
df_sample1.to_csv('sub_log1.csv', sep=',', na_rep='NaN', index = False)
#0.85906
df_sample1.country.value_counts()

# QDA / LDA

In [ ]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

In [ ]:
qda = QuadraticDiscriminantAnalysis().fit(df_train, y_label)

In [ ]:
#predict_proba
y_qda1 = qda.predict_proba(df_test)

ids = []  #list of ids
cts = []  #list of countries
for i in range(len(df_sample)):
    idx = df_sample.iloc[i, 0]
    ids += [idx] * 5
    cts += le.inverse_transform(np.argsort(y_qda1[i])[::-1])[:5].tolist()

df_sample1 = pd.DataFrame(columns=['id', 'country'])
df_sample1['id'] = ids
df_sample1['country'] = cts
df_sample1 = df_sample1.reset_index(drop=True)
df_sample1.to_csv('sub_qda1.csv', sep=',', na_rep='NaN', index = False)


df_sample1.country.value_counts()

In [ ]:
# y_qda = qda.predict(df_test)
# y_inverse_qda = le.inverse_transform(y_qda)
# y_inverse_qda

# df_sample['country'] = y_inverse_qda
# df_sample = df_sample.reset_index(drop=True)
# df_sample.to_csv('sub_qda.csv', sep=',', na_rep='NaN', index = False)
# #0.00289 #1300/1460

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [ ]:
lda = LinearDiscriminantAnalysis().fit(df_train, y_label)

In [ ]:
#predict_proba
y_lda1 = lda.predict_proba(df_test)

ids = []  #list of ids
cts = []  #list of countries
for i in range(len(df_sample)):
    idx = df_sample.iloc[i, 0]
    ids += [idx] * 5
    cts += le.inverse_transform(np.argsort(y_lda1[i])[::-1])[:5].tolist()

df_sample1 = pd.DataFrame(columns=['id', 'country'])
df_sample1['id'] = ids
df_sample1['country'] = cts
df_sample1 = df_sample1.reset_index(drop=True)
df_sample1.to_csv('sub_lda1.csv', sep=',', na_rep='NaN', index = False)
#0.85654
df_sample1.country.value_counts()

In [ ]:
# y_lda = lda.predict(df_test)
# y_inverse_lda = le.inverse_transform(y_lda)
# y_inverse_lda

# df_sample['country'] = y_inverse_lda
# df_sample = df_sample.reset_index(drop=True)
# df_sample.to_csv('sub_lda.csv', sep=',', na_rep='NaN', index = False)
# #0.68756 #1210/1460

# Naive Bayesian

In [ ]:
from sklearn.naive_bayes import MultinomialNB

In [ ]:
model_nb = MultinomialNB().fit(df_train, y_label)

In [ ]:
#predict_proba
y_nb1 = model_nb.predict_proba(df_test)

ids = []  #list of ids
cts = []  #list of countries
for i in range(len(df_sample)):
    idx = df_sample.iloc[i, 0]
    ids += [idx] * 5
    cts += le.inverse_transform(np.argsort(y_lda1[i])[::-1])[:5].tolist()

df_sample1 = pd.DataFrame(columns=['id', 'country'])
df_sample1['id'] = ids
df_sample1['country'] = cts
df_sample1 = df_sample1.reset_index(drop=True)
df_sample1.to_csv('sub_nb1.csv', sep=',', na_rep='NaN', index = False)
#0.85654
df_sample1.country.value_counts()

In [ ]:
# y_nb = model_nb.predict(df_test)
# y_inverse_nb = le.inverse_transform(y_nb)
# y_inverse_nb

# df_sample['country'] = y_inverse_nb
# df_sample = df_sample.reset_index(drop=True)
# df_sample.to_csv('sub_nb.csv', sep=',', na_rep='NaN', index = False)
# #0.54326 #1260/1460

# Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
#max_depth = 10 is the best score
tree1 = DecisionTreeClassifier(max_depth=10).fit(df_train, y_label)

In [ ]:
#predict_proba
y_tree1 = tree1.predict_proba(df_test)

ids = []  #list of ids
cts = []  #list of countries
for i in range(len(df_sample)):
    idx = df_sample.iloc[i, 0]
    ids += [idx] * 5
    cts += le.inverse_transform(np.argsort(y_tree1[i])[::-1])[:5].tolist()

df_sample1 = pd.DataFrame(columns=['id', 'country'])
df_sample1['id'] = ids
df_sample1['country'] = cts
df_sample1 = df_sample1.reset_index(drop=True)
df_sample1.to_csv('sub_tree1.csv', sep=',', na_rep='NaN', index = False)
#0.85856
df_sample1.country.value_counts()

In [ ]:
# y_tree = tree1.predict(df_test)
# y_inverse_tree = le.inverse_transform(y_tree)
# y_inverse_tree

# df_sample['country'] = y_inverse_tree
# df_sample = df_sample.reset_index(drop=True)
# df_sample.to_csv('sub_tree.csv', sep=',', na_rep='NaN', index = False)
# #0.69341 #1200/1460

# randomforest

In [ ]:
from sklearn import clone
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier

In [ ]:
model_random = RandomForestClassifier(max_depth=10, random_state=0).fit(df_train, y_label)

In [ ]:
#predict_proba
y_random1 = model_random.predict_proba(df_test)

ids = []  #list of ids
cts = []  #list of countries
for i in range(len(df_sample)):
    idx = df_sample.iloc[i, 0]
    ids += [idx] * 5
    cts += le.inverse_transform(np.argsort(y_random1[i])[::-1])[:5].tolist()

df_sample1 = pd.DataFrame(columns=['id', 'country'])
df_sample1['id'] = ids
df_sample1['country'] = cts
df_sample1 = df_sample1.reset_index(drop=True)
df_sample1.to_csv('sub_random1.csv', sep=',', na_rep='NaN', index = False)
#0.85346

In [ ]:
y_random = model_random.predict(df_test)
y_inverse_random = le.inverse_transform(y_random)
y_inverse_random

In [ ]:
df_sample['country'] = y_inverse_random
df_sample = df_sample.reset_index(drop=True)
df_sample.to_csv('sub_random.csv', sep=',', na_rep='NaN', index = False)
#0.67929 #1250/1460

In [ ]:
df_sample.country.value_counts()

# extratree

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

In [ ]:
model_extra = ExtraTreesClassifier(max_depth=4, random_state=0).fit(df_train, y_label)

In [ ]:
#predict_proba
y_extra1 = model_extra.predict_proba(df_test)

ids = []  #list of ids
cts = []  #list of countries
for i in range(len(df_sample)):
    idx = df_sample.iloc[i, 0]
    ids += [idx] * 5
    cts += le.inverse_transform(np.argsort(y_extra1[i])[::-1])[:5].tolist()

df_sample1 = pd.DataFrame(columns=['id', 'country'])
df_sample1['id'] = ids
df_sample1['country'] = cts
df_sample1 = df_sample1.reset_index(drop=True)
df_sample1.to_csv('sub_extra1.csv', sep=',', na_rep='NaN', index = False)
#0.85359

# support vector machine

In [ ]:
from sklearn import svm

In [ ]:
# #kernel
# model_svm = svm.SVC(random_state=0, probability=True).fit(df_train, y_label)

In [ ]:
# #predict_proba
# y_svm = model_svm.predict_proba(df_test)

# ids = []  #list of ids
# cts = []  #list of countries
# for i in range(len(df_sample)):
#     idx = df_sample.iloc[i, 0]
#     ids += [idx] * 5
#     cts += le.inverse_transform(np.argsort(y_svm[i])[::-1])[:5].tolist()

# df_sample1 = pd.DataFrame(columns=['id', 'country'])
# df_sample1['id'] = ids
# df_sample1['country'] = cts
# df_sample1 = df_sample1.reset_index(drop=True)
# df_sample1.to_csv('sub_svm.csv', sep=',', na_rep='NaN', index = False)
# #0.85359

In [ ]:
# #linear
# model_svm1 = svm.SVC(random_state=0, kernel= 'linear', probability=True).fit(df_train, y_label)

In [ ]:
# #predict_proba
# y_svm1 = model_svm1.predict_proba(df_test)

# ids = []  #list of ids
# cts = []  #list of countries
# for i in range(len(df_sample)):
#     idx = df_sample.iloc[i, 0]
#     ids += [idx] * 5
#     cts += le.inverse_transform(np.argsort(y_svm1[i])[::-1])[:5].tolist()

# df_sample1 = pd.DataFrame(columns=['id', 'country'])
# df_sample1['id'] = ids
# df_sample1['country'] = cts
# df_sample1 = df_sample1.reset_index(drop=True)
# df_sample1.to_csv('sub_svm1.csv', sep=',', na_rep='NaN', index = False)
# #0.85359

# XGBoost

In [ ]:
import xgboost as xgb
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from xgboost.sklearn import XGBClassifier

In [ ]:
model_xgb = xgb.XGBClassifier(max_depth=4 ,objective= 'multi:softmax', n_jobs=4 ).fit(df_train, y_label)

In [ ]:
#predict_proba
y_xgb1 = model_xgb.predict_proba(df_test)

ids = []  #list of ids
cts = []  #list of countries
for i in range(len(df_sample)):
    idx = df_sample.iloc[i, 0]
    ids += [idx] * 5
    cts += le.inverse_transform(np.argsort(y_xgb1[i])[::-1])[:5].tolist()

df_sample1 = pd.DataFrame(columns=['id', 'country'])
df_sample1['id'] = ids
df_sample1['country'] = cts
df_sample1 = df_sample1.reset_index(drop=True)
df_sample1.to_csv('sub_xgb1.csv', sep=',', na_rep='NaN', index = False)
#0.85950 w/o age
#0.86442 with age
#0.85983 w/ dummy age

In [ ]:
dtrain = xgb.DMatrix(df_train, label = y_label)

In [ ]:
y_xgb = model_xgb.predict(df_test)
y_inverse_xgb = le.inverse_transform(y_xgb)
y_inverse_xgb

In [ ]:
df_sample['country'] = y_inverse_random
df_sample = df_sample.reset_index(drop=True)
df_sample.to_csv('sub_xgb.csv', sep=',', na_rep='NaN', index = False)
#0.67929 #1250/1460

In [ ]:
df_sample.country.value_counts()

# LightGBM

In [ ]:
import lightgbm as lgb

In [ ]:
model_lgb = lgb.LGBMClassifier(max_depth=10, objective = 'multiclass', random_state=0, n_jobs=4).fit(df_train, y_label)

In [ ]:
y_vali = model_lgb.predict(df_train)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
import xgboost
from xgboost.sklearn import XGBClassifier
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(y_label, y_vali))

In [ ]:
#predict_proba
y_lgb = model_lgb.predict_proba(df_test)



ids = []  #list of ids
cts = []  #list of countries
for i in range(len(df_sample)):
    idx = df_sample.iloc[i, 0]
    ids += [idx] * 5
    cts += le.inverse_transform(np.argsort(y_lgb[i])[::-1])[:5].tolist()

df_sample1 = pd.DataFrame(columns=['id', 'country'])
df_sample1['id'] = ids
df_sample1['country'] = cts
df_sample1 = df_sample1.reset_index(drop=True)
df_sample1.to_csv('sub_lgb.csv', sep=',', na_rep='NaN', index = False)
#0.85932 w/o age
#0.86495 w/ age

In [ ]:
from sklearn import metrics
from sklearn.model_selection import cross_val_score

scores = cross_val_score(model_lgb, df_train, y_label, cv=5, scoring='neg_log_loss')
scores

# light train

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
Xtr, Xv, ytr, yv = train_test_split(df_train, y_label, test_size=0.2, random_state=0)
dtrain = lgb.Dataset(Xtr, label=ytr)
dvalid = lgb.Dataset(Xv, label=yv)
# dtest = lgb.Dataset(df_test.values)

In [ ]:
params = {}

params['application'] = 'lambdarank'
params['max_depth'] = 8
params['metric'] = 'ndcg'
# params['metric'] = 'multi_error'
# params['boosting'] = 'rf'

In [ ]:
params

In [ ]:
model_lgb1 = lgb.train(params, dtrain, valid_sets=dvalid)

In [ ]:
model_lgb1

In [ ]:
len(y_lgb1)

In [ ]:
y_lgb1 = model_lgb1.predict(df_test)



ids = []  #list of ids
cts = []  #list of countries
for i in range(len(df_sample)):
    idx = df_sample.iloc[i, 0]
    ids += [idx] * 5
    cts += le.inverse_transform(np.argsort(y_lgb1[i])[::-1])[:5].tolist()

df_sample1 = pd.DataFrame(columns=['id', 'country'])
df_sample1['id'] = ids
df_sample1['country'] = cts
df_sample1 = df_sample1.reset_index(drop=True)
df_sample1.to_csv('sub_lgb1.csv', sep=',', na_rep='NaN', index = False)


# Cross validation

### k fold stacking

In [ ]:
from sklearn.model_selection import KFold

In [ ]:
kf = KFold(ntrian, n_folds=nfolds, shuffle = kfodl_shuffle, random_state = kfold_random_state)

oof_train = np.zeros((ntrain,))
oof_test = np.zeros((ntest,))
oof_test_skf = np.empty((nfolds, ntest))

In [ ]:
for i, (train_index, cross_index) in enumerate(kf):
    x_tr, x_cr = x_train.iloc[train_index], x_train.iloc[cross_index]
    y_tr, y_cr = y_train.iloc[train_index], y_train.iloc[cross_index]
    
    clf.train(x_tr, y_tr, x_cr, y_cr)

In [ ]:
oof_train[cross_index] = clf.predict(x_cr)
oof_test_skf[i, :] = clf.predict(x_test)
oof_train[:] = oof_test_skf.mean(axis=0)


In [ ]:
import numpy as np

from sklearn.model_selection import cross_val_predict, KFold
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

X = np.array([[1, 2], [2, 4], [3, 2], [4, 4], [5, 2], [6, 4], [7, 2], [8, 4], [1, 2], [2, 4], [3, 2], [4, 4], [5, 2], [6, 4], [7, 2], [8, 4]])
Y = np.array([1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7, 8, 8])

Xtrain,Xtest,Ytrain,Ytest = train_test_split(X,Y,train_size=0.5,random_state=1)

clf = LinearDiscriminantAnalysis()
clf.fit(X, Y)

# without cross-valdidation
prediction = clf.predict(Xtest)

# with cross-valdidation
cv = KFold(n_splits=2)
prediction_cv = cross_val_predict(clf, X, Y, cv=cv)
prediction_cv